In [1]:
import torch
import pickle
from matplotlib import colors
import matplotlib.pyplot as plt
from joblib import load
from pathlib import Path
import os
import numpy as np
import sys
from config import ROOT_DIR
import pandas as pd
from libpysal.weights import Queen, Rook, DistanceBand, lat2W, WSP
from pysal.model import spreg
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from scipy.sparse import block_diag, csr_matrix
import pysal.lib as lib



sys.path.append("/Users/kevindesilva/ESC403_Project")
print(sys.path)

['/Users/kevindesilva/ESC403_Project/Notebooks', '/Users/kevindesilva/ESC403_Project', '/Users/kevindesilva/anaconda3/lib/python311.zip', '/Users/kevindesilva/anaconda3/lib/python3.11', '/Users/kevindesilva/anaconda3/lib/python3.11/lib-dynload', '', '/Users/kevindesilva/anaconda3/lib/python3.11/site-packages', '/Users/kevindesilva/anaconda3/lib/python3.11/site-packages/aeosa', '/opt/gurobi201/linux32/lib/python2.5', '/Users/kevindesilva/ESC403_Project']


In [2]:
testing = True
read_path = r"Data/trial/" if testing else r"Data/feature_engineered/"

X = load(ROOT_DIR / Path(read_path, "X_fe.pkl"))
y = load(ROOT_DIR / Path(read_path, "y_fe.pkl"))

In [3]:
print(type(X), X.shape)
print(type(y), y.shape)

<class 'torch.Tensor'> torch.Size([100, 64, 64, 15])
<class 'torch.Tensor'> torch.Size([100, 64, 64, 1])


#1: Elevation
#2: Wind direction
#3: Wind velocity
#4: Min temp
#5: Max temp
#6: Humidity
#7: Precip
#8: Drought
#9: Vegetation
#10: Population density
#11: Energy release component
#12: Previous fire mask
#13: Distance to Fire
#14: Flow Accumulation
#15: Fire Direction


#Y 1: Next Day Fire Mask


In [4]:
X_np = X.numpy()
y_np = y.numpy()


In [5]:
# Train-Test split

X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(X_np, y_np, test_size=0.2, random_state=42)


X_train = torch.tensor(X_train_np)
X_test = torch.tensor(X_test_np)
y_train = torch.tensor(y_train_np)
y_test = torch.tensor(y_test_np)

print("Trainingsdaten Formen:", X_train.shape, y_train.shape)
print("Testdaten Formen:", X_test.shape, y_test.shape)

Trainingsdaten Formen: torch.Size([80, 64, 64, 15]) torch.Size([80, 64, 64, 1])
Testdaten Formen: torch.Size([20, 64, 64, 15]) torch.Size([20, 64, 64, 1])


In [6]:
print(type(X_train_np), X_train_np.shape)
print(type(y_train_np), y_train_np.shape)

<class 'numpy.ndarray'> (80, 64, 64, 15)
<class 'numpy.ndarray'> (80, 64, 64, 1)


In [7]:
# Um das SAR Modell anzuwenden müssen die Daten 2D sein

# Um SAR-Modelle anzuwenden, reshapen wir die Daten so, dass das Raster (64x64) beibehalten wird
n_samples, height, width, n_features = X_train_np.shape

# Reshape der Eingabedaten (X) und der Zielvariablen (y)
X_train_reshaped = X_train_np.reshape(n_samples * height * width, n_features)
y_train_reshaped = y_train_np.reshape(n_samples * height * width, 1)


# Erzeuge die Gewichtsmatrix für das gesamte Dataset
# Da wir hier mit 80 64x64-Rastern arbeiten, müssen wir eine Gewichtsmatrix für das gesamte Dataset erstellen
# Das bedeutet, dass wir eine blockdiagonale Matrix erstellen müssen, die 80-mal die 64x64-Matrix enthält


# Erzeuge eine Gewichtsmatrix für ein einzelnes 64x64-Raster
w_single = lib.weights.lat2W(height, width)

w_blocks = block_diag([w_single.sparse for _ in range(n_samples)], format='csr')

# Konvertiere die blockdiagonale Matrix in ein PySAL W-Objekt
# Wir müssen die Matrix manuell in das richtige Format konvertieren
neighbors = {}
weights = {}
for i in range(n_samples):
    for key, vals in w_single.neighbors.items():
        new_key = key + i * height * width
        new_vals = [val + i * height * width for val in vals]
        neighbors[new_key] = new_vals
        weights[new_key] = w_single.weights[key]

w = lib.weights.W(neighbors, weights)

w.transform = 'r'

/var/folders/t7/b1d1yp6d79z33w_6jvk6wcr80000gn/T/ipykernel_80211/1079147782.py:32: UserWarning: The weights matrix is not fully connected: 
 There are 80 disconnected components.
  w = lib.weights.W(neighbors, weights)


In [ ]:
from tqdm import tqdm
def train_model(y_train, X_train, w, n_epochs):
    # Initialize the progress bar with the total number of epochs
    progress_bar = tqdm(total=n_epochs)
    
    for epoch in range(n_epochs):
        # Simulate the training step (this would be your actual training code)
        sar_model = spreg.ML_Lag(y_train, X_train, w=w, name_y='y', name_x=['X' + str(i) for i in range(X_train.shape[1])])
        
        # Update the progress bar
        progress_bar.update(1)
    
    # Close the progress bar after training is complete
    progress_bar.close()

# Assuming y_train_reshaped and X_train_reshaped are your training data and w is your weights
n_epochs = 100  # Total number of training epochs
train_model(y_train_reshaped, X_train_reshaped, w, n_epochs)


#sar_model = spreg.ML_Lag(y_train_reshaped, X_train_reshaped, w=w, name_y='y', name_x=['X' + str(i) for i in range(n_features)])

# Modell-Details anzeigen

#print(sar_model.summary)

  0%|          | 0/100 [00:00<?, ?it/s]

In [39]:
print(type(X_train_reshaped), X_train_reshaped.shape)
print(type(y_train_reshaped), y_train_reshaped.shape)

print(type(w))

<class 'numpy.ndarray'> (327680, 15)
<class 'numpy.ndarray'> (327680, 1)
<class 'libpysal.weights.weights.W'>
